In [ ]:
# import libraries, see https://www.rsgislib.org/index.html
import rsgislib, os
from rsgislib import imagefilter, imageutils

# STEP 1: filter drone orthomosaic

# define input drone image - designed to work with RGB orthoimagery
inputImage = '/Users/Andy/Documents/Zanzibar/IVCC/SIS/mapping_quality_study/data/orthoimages/ndagaa/ndagaa_b2_6_5_21.tif'

# if needed, set no data value, i.e. 255
rsgislib.imageutils.popImageStats(inputImage, usenodataval=True, nodataval=255, calcpyramids=False)

# define output filtered image, filetype KEA
outImgFile = 'my/input/image.kea'

# apply 3x3 median filter and calculate image statistics
imagefilter.applyMedianFilter(inputImage, outImgFile, 3, "KEA", rsgislib.TYPE_8INT)
rsgislib.imageutils.popImageStats(outImgFile, usenodataval=True, nodataval=255, calcpyramids=False)



In [ ]:
# STEP 2: perform image segmentation

# import libraries
from rsgislib.segmentation import segutils

# define input image as the median filtered image from step 1
inImage=outImgFile
outputClumps=inImage.replace('.kea','_clumps2.kea')

start = time.time()

outputMeanImg = inImage.replace('.kea','_clumps2_mean.kea')

# apply Shepherd Segmentation

segutils.runShepherdSegmentation(inImage,
                                 outputClumps,
                                 outputMeanImg,
                                 tmpath='./',
                                 numClusters=60,
                                 bands=[1,2,3],
                                 minPxls=50,
                                 distThres=100,
                                 sampling=100, kmMaxIter=200)


In [ ]:
# STEP 3: calculate image statistics and attach to the raster attribute table (RAT)

from rsgislib import rastergis
clumps=outputClumps


bs = []
bs.append(rastergis.BandAttStats(band=1, minField='redMin', maxField='redMax', meanField='redMean', stdDevField='redStdDev'))
bs.append(rastergis.BandAttStats(band=2, minField='greenMin', maxField='greenMax', meanField='greenMean', stdDevField='greenStdDev'))
bs.append(rastergis.BandAttStats(band=3, minField='blueMin', maxField='blueMax', meanField='blueMean', stdDevField='blueStdDev'))
rastergis.populateRATWithStats(inImage, clumps, bs)

In [ ]:
# STEP 4: import vector training data and attach to RAT

from rsgislib.rastergis import ratutils

root='/path/to/training_features/folder/'

# Create list of training data and populate the RAT
# comment out lines below where training data does not exist
# add more training data classes if necessary
classesDict = dict()
classesDict['OpenWater'] = [1, root+'open_water.shp']
classesDict['SunGlint'] = [2, root+'sun_glint.shp']
classesDict['EmergentVeg'] = [3, root+'emerge_veg.shp']
classesDict['DenseCanopy'] = [4, root+'canopy.shp']
classesDict['TrackRoad'] = [5, root+'road.shp']
classesDict['TilledSoil'] = [6, root+'tilled_soil.shp']
classesDict['Crops'] = [7, root+'crops.shp']
classesDict['RoofMetal'] = [8, root+'roof_metal.shp']
classesDict['RoofBlue'] = [9, root+'roof_blue.shp']
classesDict['RoofRed'] = [10, './training_features/roof_red.shp']
classesDict['Shadow'] = [11, root+'shadow.shp']
tmpPath = './tmp'

# define names of ourput columns in the RAT
classesIntColIn = 'ClassInt'
classesNameCol = 'ClassStr'

# populate RAT with training data
ratutils.populateClumpsWithClassTraining(clumps, classesDict, tmpPath,
classesIntColIn, classesNameCol)

In [ ]:
from rsgislib import rastergis

# define which columns to use as independent/predicting variables
variables=['redMin', 'redMax', 'redMean', 'redStdDev','greenMin', 'greenMax', 'greenMean', 'greenStdDev', 'blueMin','blueMax', 'blueMean', 'blueStdDev']

# export the RAT as a csv
outfile=clumps.replace('.kea','_RAT_reduced.csv')
rsgislib.rastergis.export2Ascii(clumps, outfile, variables)

In [ ]:
# STEP 5: Check the independent variables for co-linearity

import pandas as pd
from pandas import DataFrame

# convert the csv to a pandas dataframe
df=pd.read_csv(outfile)
df=df[df.FID!=0]
df=df.drop('FID', axis=1)

# generate a pair-wise correlation
corr=df.corr()
corr2=corr.round(decimals=2)

# function to style significant (<-0.8 or >0.8) correlation values
def color_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val >= 0.8 or val <= -0.8 else 'black'
    return 'color: %s' % color

# produce the colour-coded table
corr_coloured = corr2.style.applymap(color_red)
corr_coloured

In [ ]:
# STEP 6: Create a mask to ignore background values

# read the RAT using gdal
ratDataset = gdal.Open(clumps, gdal.GA_Update)

r = rat.readColumn(ratDataset, 'redMean')
g = rat.readColumn(ratDataset, 'greenMean')
b = rat.readColumn(ratDataset, 'blueMean')
roi = numpy.where((r>254)&(g>254)&(b>254), 1, 0)

# add the mask, or region of interest, to the RAT
rat.writeColumn(ratDataset, "roi", roi)
ratDataset = None

In [ ]:
# STEP 7: Apply the supervised classification 

from rsgislib import rastergis
from rsgislib.rastergis import ratutils
from rsgislib.classification import classratutils
from rsgislib import classification
import osgeo.gdal as gdal
from rios import rat
import numpy
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV

# Ensure there are a minimum number of training samples and
# balance so there are the same number for each class

# define the column for class numbers
classesIntCol = 'ClassInt'
# balance training samples based on min/max samples
classratutils.balanceSampleTrainingRandom(clumps, classesIntColIn, classesIntCol, 10, 200)

# define parameters for conducting the grid search of machine learning parameters
classParameters = {'n_estimators':[10,100,500], 'max_features':[2,3,4]}

# define the classifier type 
gSearch = GridSearchCV(ExtraTreesClassifier(), classParameters)
preProcessing = None

# perform the grid search
classifier = classratutils.findClassifierParameters(clumps, classesIntCol, variables, preProcessor=None, gridSearch=gSearch)

# define a dictionary of colours for each class type to apply to the classification output 
classColours = dict()

classColours['OpenWater'] = [15,128,255]
classColours['SunGlint'] = [32,255,255]
classColours['EmergentVeg'] = [161,179,30]
classColours['DenseCanopy'] = [108,156,35]
classColours['TrackRoad'] = [255,255,102]
classColours['TilledSoil'] = [128,64,2]
classColours['Crops'] = [102,255,102]
classColours['RoofMetal'] = [255,255,255]
classColours['RoofBlue'] = [253,102,102]
classColours['RoofRed'] = [102,102,255]
classColours['Shadow'] = [60,60,60]


# run the classifier with the best parameter choice following the grid search
outClassIntCol = 'OutClass'
outClassStrCol = 'OutClassName'
classratutils.classifyWithinRAT(clumps, classesIntCol, classesNameCol, 
                                variables, classifier=classifier, 
                                outColInt=outClassIntCol,
                                outColStr=outClassStrCol, classColours=classColours,
                                preProcessor=preProcessing, roiCol="roi", roiVal=0)    

In [ ]:
# STEP 8: Extract thematic classification image from the RAT

# define the filename of the final output classification
outClassImg = clumps.replace('.kea','_extrees_class.tif')

# commit the output classes to the output file
classification.collapseClasses(clumps, outClassImg, 'GTiff', 'OutClassName', 'OutClass')                                    